# 🤖 Agentic RAG Implementation

## Autonomous Agents with Iterative Reasoning

This notebook demonstrates an **Agentic RAG system** with:
- 🧠 Intelligent Planning Agents
- 🛠️ Multi-Tool Integration 
- 🔄 Iterative Execution & Refinement
- 🤔 Self-Reflection & Validation
- 🎯 Goal-Oriented Problem Solving
- 🤝 Multi-Agent Coordination

### Key Benefits of Agentic Architecture
- **Autonomy**: Agents make independent decisions
- **Adaptability**: Dynamic plan adjustment based on results
- **Tool Mastery**: Intelligent tool selection and usage
- **Self-Improvement**: Learn from successes and failures
- **Complex Reasoning**: Break down multi-step problems

In [1]:
# Install required packages
!pip install sentence-transformers faiss-cpu google-generativeai numpy requests beautifulsoup4 python-dotenv sympy matplotlib seaborn pandas

     |████████████████████████████████| 8.6 MB 4.2 MB/s eta 0:00:01
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     |████████████████████████████████| 12.3 MB 9.5 MB/s eta 0:00:01
  Using cached contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (325 kB)
     |████████████████████████████████| 4.8 MB 9.0 MB/s eta 0:00:01
  Using cached kiwisolver-1.4.8-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
You should consider upgrading via the '/home/mohdasimkhan/.pyenv/versions/3.10.2/envs/rags/bin/python -m pip install --upgrade pip' command.


In [2]:
# Import libraries
import numpy as np
import json
import re
import os
import time
import uuid
import requests
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional, Any, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum
from datetime import datetime, timedelta
import asyncio
from concurrent.futures import ThreadPoolExecutor

from sentence_transformers import SentenceTransformer
import faiss
import google.generativeai as genai
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv()
print("🤖 Libraries imported successfully for Agentic RAG!")

/home/mohdasimkhan/.pyenv/versions/3.10.2/envs/rags/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🤖 Libraries imported successfully for Agentic RAG!


## 🏗️ Core Agentic Data Structures

Define the foundational structures for our agentic system:

In [3]:
# Enums for agentic system
class AgentType(Enum):
    PLANNER = "planner"
    EXECUTOR = "executor"
    REFLECTOR = "reflector"
    COORDINATOR = "coordinator"
    SPECIALIST = "specialist"

class TaskType(Enum):
    SEARCH = "search"
    CALCULATE = "calculate"
    ANALYZE = "analyze"
    SYNTHESIZE = "synthesize"
    VERIFY = "verify"

class PlanStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"
    REVISED = "revised"

class ToolType(Enum):
    WEB_SEARCH = "web_search"
    CALCULATOR = "calculator"
    DATABASE = "database"
    RETRIEVER = "retriever"
    ANALYZER = "analyzer"

# Core agentic data structures
@dataclass
class Task:
    id: str
    description: str
    task_type: TaskType
    tool_required: ToolType
    parameters: Dict[str, Any] = field(default_factory=dict)
    status: PlanStatus = PlanStatus.PENDING
    result: Optional[Any] = None
    confidence: float = 0.0
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class Plan:
    id: str
    goal: str
    tasks: List[Task]
    status: PlanStatus = PlanStatus.PENDING
    current_step: int = 0
    results: List[Any] = field(default_factory=list)
    created_at: datetime = field(default_factory=datetime.now)
    revised_count: int = 0

@dataclass
class AgentMessage:
    sender: str
    receiver: str
    content: Any
    message_type: str
    timestamp: datetime = field(default_factory=datetime.now)

@dataclass
class AgenticResponse:
    query: str
    final_answer: str
    execution_plan: Plan
    tool_usage: Dict[str, int]
    confidence_score: float
    reasoning_steps: List[str]
    processing_time: float
    iterations: int

print("🏗️ Agentic data structures defined!")

🏗️ Agentic data structures defined!


## 🛠️ Tool System Architecture

Create a comprehensive tool system for agents:

In [4]:
# Base tool class
class BaseTool(ABC):
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.usage_count = 0
        self.success_rate = 1.0
        self.avg_execution_time = 0.0
    
    @abstractmethod
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        """Execute tool and return (result, confidence)"""
        pass
    
    def update_stats(self, execution_time: float, success: bool):
        self.usage_count += 1
        
        # Update success rate
        if self.usage_count == 1:
            self.success_rate = 1.0 if success else 0.0
        else:
            self.success_rate = ((self.success_rate * (self.usage_count - 1)) + 
                               (1.0 if success else 0.0)) / self.usage_count
        
        # Update average execution time
        self.avg_execution_time = ((self.avg_execution_time * (self.usage_count - 1)) + 
                                  execution_time) / self.usage_count
    
    def get_stats(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'usage_count': self.usage_count,
            'success_rate': self.success_rate,
            'avg_execution_time': self.avg_execution_time
        }

# Web Search Tool
class WebSearchTool(BaseTool):
    def __init__(self):
        super().__init__(
            name="WebSearch",
            description="Search the web for current information"
        )
        self.search_history = []
    
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        start_time = time.time()
        
        try:
            query = parameters.get('query', '')
            max_results = parameters.get('max_results', 3)
            
            # Simulate web search (in real implementation, use actual search API)
            results = self._simulate_search(query, max_results)
            
            execution_time = time.time() - start_time
            self.update_stats(execution_time, True)
            
            return results, 0.8
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.update_stats(execution_time, False)
            return f"Search failed: {str(e)}", 0.0
    
    def _simulate_search(self, query: str, max_results: int) -> List[Dict[str, str]]:
        # Simulated search results
        search_results = [
            {
                "title": f"Result for '{query}' - Article 1",
                "snippet": f"This article discusses {query} and provides comprehensive information about the topic.",
                "url": "https://example.com/article1"
            },
            {
                "title": f"Understanding {query} - Complete Guide",
                "snippet": f"A detailed guide covering all aspects of {query} with practical examples.",
                "url": "https://example.com/guide"
            },
            {
                "title": f"Latest Research on {query}",
                "snippet": f"Recent studies and findings related to {query} from leading researchers.",
                "url": "https://example.com/research"
            }
        ]
        
        return search_results[:max_results]

# Calculator Tool
class CalculatorTool(BaseTool):
    def __init__(self):
        super().__init__(
            name="Calculator",
            description="Perform mathematical calculations and symbolic math"
        )
    
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        start_time = time.time()
        
        try:
            expression = parameters.get('expression', '')
            calc_type = parameters.get('type', 'numeric')  # numeric, symbolic, equation
            
            if calc_type == 'symbolic':
                result = self._symbolic_calculation(expression)
            elif calc_type == 'equation':
                result = self._solve_equation(expression)
            else:
                result = self._numeric_calculation(expression)
            
            execution_time = time.time() - start_time
            self.update_stats(execution_time, True)
            
            return result, 0.95
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.update_stats(execution_time, False)
            return f"Calculation failed: {str(e)}", 0.0
    
    def _numeric_calculation(self, expression: str) -> float:
        # Safe evaluation of mathematical expressions
        allowed_names = {
            k: v for k, v in np.__dict__.items() if not k.startswith("__")
        }
        allowed_names.update({
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow
        })
        
        return eval(expression, {"__builtins__": {}}, allowed_names)
    
    def _symbolic_calculation(self, expression: str) -> str:
        # Symbolic math using SymPy
        x, y, z = sp.symbols('x y z')
        expr = sp.sympify(expression)
        simplified = sp.simplify(expr)
        return str(simplified)
    
    def _solve_equation(self, equation: str) -> List[str]:
        # Solve equations
        x = sp.Symbol('x')
        eq = sp.Eq(sp.sympify(equation.split('=')[0]), sp.sympify(equation.split('=')[1]))
        solutions = sp.solve(eq, x)
        return [str(sol) for sol in solutions]

# Database Query Tool
class DatabaseTool(BaseTool):
    def __init__(self):
        super().__init__(
            name="Database",
            description="Query structured data and knowledge bases"
        )
        self.knowledge_base = self._create_sample_kb()
    
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        start_time = time.time()
        
        try:
            query = parameters.get('query', '')
            query_type = parameters.get('type', 'search')  # search, filter, aggregate
            
            if query_type == 'search':
                result = self._search_knowledge(query)
            elif query_type == 'filter':
                result = self._filter_data(parameters)
            else:
                result = self._aggregate_data(parameters)
            
            execution_time = time.time() - start_time
            self.update_stats(execution_time, True)
            
            return result, 0.85
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.update_stats(execution_time, False)
            return f"Database query failed: {str(e)}", 0.0
    
    def _create_sample_kb(self) -> Dict[str, Any]:
        return {
            'companies': [
                {'name': 'Apple', 'sector': 'Technology', 'founded': 1976, 'revenue': 394.3},
                {'name': 'Microsoft', 'sector': 'Technology', 'founded': 1975, 'revenue': 211.9},
                {'name': 'Amazon', 'sector': 'E-commerce', 'founded': 1994, 'revenue': 513.9}
            ],
            'technologies': [
                {'name': 'Machine Learning', 'category': 'AI', 'maturity': 'Mature'},
                {'name': 'Quantum Computing', 'category': 'Computing', 'maturity': 'Emerging'},
                {'name': 'Blockchain', 'category': 'Distributed', 'maturity': 'Growing'}
            ]
        }
    
    def _search_knowledge(self, query: str) -> List[Dict[str, Any]]:
        results = []
        query_lower = query.lower()
        
        for category, items in self.knowledge_base.items():
            for item in items:
                # Simple text matching
                if any(query_lower in str(value).lower() for value in item.values()):
                    results.append({'category': category, 'data': item})
        
        return results
    
    def _filter_data(self, parameters: Dict[str, Any]) -> List[Dict[str, Any]]:
        # Implement filtering logic
        return []
    
    def _aggregate_data(self, parameters: Dict[str, Any]) -> Dict[str, Any]:
        # Implement aggregation logic
        return {}

# Retrieval Tool
class RetrievalTool(BaseTool):
    def __init__(self):
        super().__init__(
            name="Retrieval",
            description="Retrieve relevant documents from knowledge base"
        )
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.documents = []
        self.index = None
        self._setup_knowledge_base()
    
    def _setup_knowledge_base(self):
        # Sample documents
        docs = [
            "Artificial Intelligence is transforming industries through automation and intelligent decision-making.",
            "Machine Learning algorithms learn patterns from data to make predictions without explicit programming.",
            "Quantum Computing leverages quantum mechanics to solve complex problems exponentially faster.",
            "Cloud Computing provides scalable, on-demand access to computing resources over the internet.",
            "Digital transformation integrates digital technology into all business areas, changing operations."
        ]
        
        self.documents = docs
        embeddings = self.embedding_model.encode(docs)
        
        # Create FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings.astype('float32'))
    
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        start_time = time.time()
        
        try:
            query = parameters.get('query', '')
            top_k = parameters.get('top_k', 3)
            
            query_embedding = self.embedding_model.encode([query])
            faiss.normalize_L2(query_embedding)
            
            scores, indices = self.index.search(query_embedding.astype('float32'), top_k)
            
            results = []
            for score, idx in zip(scores[0], indices[0]):
                results.append({
                    'document': self.documents[idx],
                    'score': float(score)
                })
            
            execution_time = time.time() - start_time
            self.update_stats(execution_time, True)
            
            return results, 0.9
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.update_stats(execution_time, False)
            return f"Retrieval failed: {str(e)}", 0.0

# Analyzer Tool
class AnalyzerTool(BaseTool):
    def __init__(self):
        super().__init__(
            name="Analyzer",
            description="Analyze data and extract insights"
        )
    
    def execute(self, parameters: Dict[str, Any]) -> Tuple[Any, float]:
        start_time = time.time()
        
        try:
            data = parameters.get('data', [])
            analysis_type = parameters.get('type', 'summary')  # summary, trend, correlation
            
            if analysis_type == 'summary':
                result = self._summarize_data(data)
            elif analysis_type == 'trend':
                result = self._analyze_trends(data)
            else:
                result = self._find_correlations(data)
            
            execution_time = time.time() - start_time
            self.update_stats(execution_time, True)
            
            return result, 0.8
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.update_stats(execution_time, False)
            return f"Analysis failed: {str(e)}", 0.0
    
    def _summarize_data(self, data: List[Any]) -> Dict[str, Any]:
        if not data:
            return {"summary": "No data to analyze"}
        
        return {
            "count": len(data),
            "first_item": str(data[0]) if data else None,
            "last_item": str(data[-1]) if data else None,
            "summary": f"Dataset contains {len(data)} items"
        }
    
    def _analyze_trends(self, data: List[Any]) -> Dict[str, Any]:
        return {"trend": "Analysis would identify patterns and trends in the data"}
    
    def _find_correlations(self, data: List[Any]) -> Dict[str, Any]:
        return {"correlations": "Analysis would find relationships between variables"}

print("🛠️ Tool system created with 5 specialized tools!")

🛠️ Tool system created with 5 specialized tools!


## 🧠 Agent Base Classes

Define base classes for autonomous agents:

In [5]:
# Base Agent class
class BaseAgent(ABC):
    def __init__(self, name: str, agent_type: AgentType):
        self.name = name
        self.agent_type = agent_type
        self.agent_id = str(uuid.uuid4())[:8]
        self.created_at = datetime.now()
        self.total_tasks = 0
        self.successful_tasks = 0
        self.message_queue = []
        self.memory = {}
    
    @abstractmethod
    def process(self, input_data: Any) -> Any:
        pass
    
    def send_message(self, receiver: str, content: Any, message_type: str):
        message = AgentMessage(
            sender=self.name,
            receiver=receiver,
            content=content,
            message_type=message_type
        )
        return message
    
    def receive_message(self, message: AgentMessage):
        self.message_queue.append(message)
    
    def update_stats(self, success: bool):
        self.total_tasks += 1
        if success:
            self.successful_tasks += 1
    
    def get_success_rate(self) -> float:
        if self.total_tasks == 0:
            return 0.0
        return self.successful_tasks / self.total_tasks
    
    def get_info(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'type': self.agent_type.value,
            'id': self.agent_id,
            'total_tasks': self.total_tasks,
            'success_rate': self.get_success_rate(),
            'created_at': self.created_at
        }

print("🧠 Base agent class defined!")

🧠 Base agent class defined!


## 📋 Action Planner Agent

Intelligent agent that creates execution plans:

In [6]:
class ActionPlannerAgent(BaseAgent):
    def __init__(self):
        super().__init__("ActionPlanner", AgentType.PLANNER)
        
        # Planning patterns and strategies
        self.planning_patterns = {
            'search_and_analyze': [
                (TaskType.SEARCH, ToolType.WEB_SEARCH),
                (TaskType.ANALYZE, ToolType.ANALYZER),
                (TaskType.SYNTHESIZE, ToolType.RETRIEVER)
            ],
            'calculate_and_verify': [
                (TaskType.CALCULATE, ToolType.CALCULATOR),
                (TaskType.VERIFY, ToolType.DATABASE),
                (TaskType.SYNTHESIZE, ToolType.ANALYZER)
            ],
            'research_deep': [
                (TaskType.SEARCH, ToolType.WEB_SEARCH),
                (TaskType.SEARCH, ToolType.RETRIEVER),
                (TaskType.SEARCH, ToolType.DATABASE),
                (TaskType.ANALYZE, ToolType.ANALYZER),
                (TaskType.SYNTHESIZE, ToolType.RETRIEVER)
            ]
        }
    
    def process(self, query: str) -> Plan:
        self.update_stats(True)  # Planning rarely fails
        
        plan_id = str(uuid.uuid4())[:8]
        
        # Analyze query complexity and type
        query_analysis = self._analyze_query(query)
        
        # Select planning pattern
        pattern = self._select_pattern(query_analysis)
        
        # Create tasks
        tasks = self._create_tasks(query, pattern, query_analysis)
        
        plan = Plan(
            id=plan_id,
            goal=query,
            tasks=tasks
        )
        
        # Store planning context in memory
        self.memory[plan_id] = {
            'query_analysis': query_analysis,
            'selected_pattern': pattern,
            'original_query': query
        }
        
        return plan
    
    def revise_plan(self, original_plan: Plan, failed_task: Task, failure_reason: str) -> Plan:
        """Revise plan when a task fails"""
        print(f"🔄 Revising plan due to failed task: {failed_task.description}")
        
        # Get original context
        context = self.memory.get(original_plan.id, {})
        
        # Create alternative tasks
        alternative_tasks = self._create_alternative_tasks(
            failed_task, failure_reason, context
        )
        
        # Create revised plan
        revised_tasks = original_plan.tasks[:original_plan.current_step] + alternative_tasks
        
        revised_plan = Plan(
            id=str(uuid.uuid4())[:8],
            goal=original_plan.goal,
            tasks=revised_tasks,
            current_step=original_plan.current_step,
            results=original_plan.results.copy(),
            revised_count=original_plan.revised_count + 1
        )
        
        return revised_plan
    
    def _analyze_query(self, query: str) -> Dict[str, Any]:
        query_lower = query.lower()
        
        analysis = {
            'length': len(query.split()),
            'complexity': 'simple',
            'requires_calculation': False,
            'requires_search': False,
            'requires_analysis': False,
            'domain': 'general'
        }
        
        # Check for calculation keywords
        calc_keywords = ['calculate', 'compute', 'solve', 'equation', 'math', 'formula']
        if any(keyword in query_lower for keyword in calc_keywords):
            analysis['requires_calculation'] = True
        
        # Check for search keywords
        search_keywords = ['search', 'find', 'lookup', 'what is', 'who is', 'when did']
        if any(keyword in query_lower for keyword in search_keywords):
            analysis['requires_search'] = True
        
        # Check for analysis keywords
        analysis_keywords = ['analyze', 'compare', 'evaluate', 'assess', 'impact']
        if any(keyword in query_lower for keyword in analysis_keywords):
            analysis['requires_analysis'] = True
        
        # Determine complexity
        complexity_score = 0
        if analysis['length'] > 10:
            complexity_score += 1
        if analysis['requires_calculation']:
            complexity_score += 1
        if analysis['requires_analysis']:
            complexity_score += 2
        
        if complexity_score <= 1:
            analysis['complexity'] = 'simple'
        elif complexity_score <= 3:
            analysis['complexity'] = 'medium'
        else:
            analysis['complexity'] = 'complex'
        
        return analysis
    
    def _select_pattern(self, analysis: Dict[str, Any]) -> str:
        if analysis['requires_calculation']:
            return 'calculate_and_verify'
        elif analysis['complexity'] == 'complex':
            return 'research_deep'
        else:
            return 'search_and_analyze'
    
    def _create_tasks(self, query: str, pattern: str, analysis: Dict[str, Any]) -> List[Task]:
        tasks = []
        pattern_tasks = self.planning_patterns[pattern]
        
        for i, (task_type, tool_type) in enumerate(pattern_tasks):
            task_id = f"task_{i+1}"
            
            # Customize task based on type
            if task_type == TaskType.SEARCH:
                description = f"Search for information about: {query}"
                parameters = {'query': query, 'max_results': 3}
            elif task_type == TaskType.CALCULATE:
                description = f"Perform calculations related to: {query}"
                parameters = {'expression': query, 'type': 'numeric'}
            elif task_type == TaskType.ANALYZE:
                description = f"Analyze gathered information about: {query}"
                parameters = {'type': 'summary'}
            elif task_type == TaskType.VERIFY:
                description = f"Verify information accuracy for: {query}"
                parameters = {'query': query, 'type': 'search'}
            else:  # SYNTHESIZE
                description = f"Synthesize final answer for: {query}"
                parameters = {'query': query, 'top_k': 3}
            
            task = Task(
                id=task_id,
                description=description,
                task_type=task_type,
                tool_required=tool_type,
                parameters=parameters
            )
            
            tasks.append(task)
        
        return tasks
    
    def _create_alternative_tasks(self, failed_task: Task, failure_reason: str, context: Dict[str, Any]) -> List[Task]:
        """Create alternative tasks when original task fails"""
        alternative_tasks = []
        
        # Try different tool for same task type
        if failed_task.task_type == TaskType.SEARCH:
            if failed_task.tool_required == ToolType.WEB_SEARCH:
                # Try retrieval instead
                alt_task = Task(
                    id=f"alt_{failed_task.id}",
                    description=f"Retrieve information (alternative): {failed_task.description}",
                    task_type=TaskType.SEARCH,
                    tool_required=ToolType.RETRIEVER,
                    parameters=failed_task.parameters
                )
                alternative_tasks.append(alt_task)
        
        # Add verification task
        verify_task = Task(
            id=f"verify_{failed_task.id}",
            description=f"Verify alternative approach for: {failed_task.description}",
            task_type=TaskType.VERIFY,
            tool_required=ToolType.DATABASE,
            parameters={'query': failed_task.parameters.get('query', ''), 'type': 'search'}
        )
        alternative_tasks.append(verify_task)
        
        return alternative_tasks

# Test planner
planner = ActionPlannerAgent()
test_plan = planner.process("What is the impact of artificial intelligence on healthcare?")

print(f"📋 Test plan created:")
print(f"   Goal: {test_plan.goal}")
print(f"   Tasks: {len(test_plan.tasks)}")
for i, task in enumerate(test_plan.tasks[:3], 1):
    print(f"   {i}. {task.description} (Tool: {task.tool_required.value})")
print("✅ Action Planner Agent ready!")

📋 Test plan created:
   Goal: What is the impact of artificial intelligence on healthcare?
   Tasks: 3
   1. Search for information about: What is the impact of artificial intelligence on healthcare? (Tool: web_search)
   2. Analyze gathered information about: What is the impact of artificial intelligence on healthcare? (Tool: analyzer)
   3. Synthesize final answer for: What is the impact of artificial intelligence on healthcare? (Tool: retriever)
✅ Action Planner Agent ready!


## ⚡ Tool Executor Agent

Agent that executes tasks using available tools:

In [7]:
class ToolExecutorAgent(BaseAgent):
    def __init__(self):
        super().__init__("ToolExecutor", AgentType.EXECUTOR)
        
        # Initialize all tools
        self.tools = {
            ToolType.WEB_SEARCH: WebSearchTool(),
            ToolType.CALCULATOR: CalculatorTool(),
            ToolType.DATABASE: DatabaseTool(),
            ToolType.RETRIEVER: RetrievalTool(),
            ToolType.ANALYZER: AnalyzerTool()
        }
        
        self.execution_history = []
        self.tool_usage_stats = {tool_type: 0 for tool_type in ToolType}
    
    def process(self, task: Task) -> Task:
        """Execute a single task"""
        print(f"⚡ Executing: {task.description}")
        
        start_time = time.time()
        task.status = PlanStatus.IN_PROGRESS
        
        try:
            # Get the appropriate tool
            tool = self.tools.get(task.tool_required)
            if not tool:
                raise ValueError(f"Tool not available: {task.tool_required}")
            
            # Execute tool with task parameters
            result, confidence = tool.execute(task.parameters)
            
            # Update task with results
            task.result = result
            task.confidence = confidence
            task.status = PlanStatus.COMPLETED
            
            # Update statistics
            self.update_stats(True)
            self.tool_usage_stats[task.tool_required] += 1
            
            execution_time = time.time() - start_time
            
            # Log execution
            self.execution_history.append({
                'task_id': task.id,
                'tool_used': task.tool_required.value,
                'execution_time': execution_time,
                'confidence': confidence,
                'success': True,
                'timestamp': datetime.now()
            })
            
            print(f"   ✅ Completed in {execution_time:.2f}s (Confidence: {confidence:.2f})")
            
        except Exception as e:
            # Handle execution failure
            task.result = f"Execution failed: {str(e)}"
            task.confidence = 0.0
            task.status = PlanStatus.FAILED
            
            self.update_stats(False)
            
            execution_time = time.time() - start_time
            
            self.execution_history.append({
                'task_id': task.id,
                'tool_used': task.tool_required.value,
                'execution_time': execution_time,
                'confidence': 0.0,
                'success': False,
                'error': str(e),
                'timestamp': datetime.now()
            })
            
            print(f"   ❌ Failed after {execution_time:.2f}s: {str(e)}")
        
        return task
    
    def execute_plan(self, plan: Plan) -> Plan:
        """Execute all tasks in a plan sequentially"""
        print(f"🎯 Executing plan: {plan.goal}")
        
        plan.status = PlanStatus.IN_PROGRESS
        
        for i, task in enumerate(plan.tasks):
            plan.current_step = i
            
            # Execute task
            executed_task = self.process(task)
            
            # Store result
            plan.results.append(executed_task.result)
            
            # Update plan task
            plan.tasks[i] = executed_task
            
            # Check if task failed and needs plan revision
            if executed_task.status == PlanStatus.FAILED:
                plan.status = PlanStatus.FAILED
                break
        
        if plan.status != PlanStatus.FAILED:
            plan.status = PlanStatus.COMPLETED
            plan.current_step = len(plan.tasks)
        
        return plan
    
    def get_tool_stats(self) -> Dict[str, Any]:
        """Get statistics about tool usage"""
        tool_stats = {}
        
        for tool_type, tool in self.tools.items():
            tool_stats[tool_type.value] = {
                'usage_count': self.tool_usage_stats[tool_type],
                'tool_stats': tool.get_stats()
            }
        
        return tool_stats
    
    def get_execution_summary(self) -> Dict[str, Any]:
        """Get summary of all executions"""
        if not self.execution_history:
            return {'total_executions': 0}
        
        successful_executions = [e for e in self.execution_history if e['success']]
        
        return {
            'total_executions': len(self.execution_history),
            'successful_executions': len(successful_executions),
            'success_rate': len(successful_executions) / len(self.execution_history),
            'avg_execution_time': np.mean([e['execution_time'] for e in self.execution_history]),
            'avg_confidence': np.mean([e['confidence'] for e in successful_executions]) if successful_executions else 0
        }

# Test executor
executor = ToolExecutorAgent()
test_task = Task(
    id="test_1",
    description="Search for AI information",
    task_type=TaskType.SEARCH,
    tool_required=ToolType.WEB_SEARCH,
    parameters={'query': 'artificial intelligence', 'max_results': 2}
)

executed_task = executor.process(test_task)
print(f"✅ Tool Executor Agent ready!")
print(f"   Test execution status: {executed_task.status.value}")
print(f"   Test confidence: {executed_task.confidence:.2f}")

⚡ Executing: Search for AI information
   ✅ Completed in 0.00s (Confidence: 0.80)
✅ Tool Executor Agent ready!
   Test execution status: completed
   Test confidence: 0.80


## 🤔 Self-Reflector Agent

Agent that validates results and provides feedback:

In [8]:
class SelfReflectorAgent(BaseAgent):
    def __init__(self):
        super().__init__("SelfReflector", AgentType.REFLECTOR)
        
        # Validation criteria
        self.validation_criteria = {
            'confidence_threshold': 0.6,
            'completeness_threshold': 0.7,
            'consistency_threshold': 0.8,
            'relevance_threshold': 0.7
        }
        
        self.reflection_history = []
    
    def process(self, plan: Plan) -> Dict[str, Any]:
        """Validate plan execution and provide reflection"""
        print(f"🤔 Reflecting on plan execution...")
        
        reflection = {
            'plan_id': plan.id,
            'overall_success': False,
            'confidence_score': 0.0,
            'completeness_score': 0.0,
            'consistency_score': 0.0,
            'relevance_score': 0.0,
            'issues_found': [],
            'recommendations': [],
            'should_revise': False
        }
        
        # Validate each completed task
        completed_tasks = [t for t in plan.tasks if t.status == PlanStatus.COMPLETED]
        failed_tasks = [t for t in plan.tasks if t.status == PlanStatus.FAILED]
        
        if not completed_tasks:
            reflection['issues_found'].append("No tasks completed successfully")
            reflection['should_revise'] = True
            self.reflection_history.append(reflection)
            return reflection
        
        # Calculate confidence score
        confidence_scores = [t.confidence for t in completed_tasks]
        reflection['confidence_score'] = np.mean(confidence_scores)
        
        # Calculate completeness score
        total_tasks = len(plan.tasks)
        completed_count = len(completed_tasks)
        reflection['completeness_score'] = completed_count / total_tasks
        
        # Calculate consistency score
        reflection['consistency_score'] = self._assess_consistency(completed_tasks)
        
        # Calculate relevance score
        reflection['relevance_score'] = self._assess_relevance(plan.goal, completed_tasks)
        
        # Check validation criteria
        issues = self._check_validation_criteria(reflection)
        reflection['issues_found'] = issues
        
        # Generate recommendations
        recommendations = self._generate_recommendations(reflection, failed_tasks)
        reflection['recommendations'] = recommendations
        
        # Determine if revision is needed
        reflection['should_revise'] = self._should_revise_plan(reflection)
        
        # Overall success assessment
        reflection['overall_success'] = (
            reflection['confidence_score'] >= self.validation_criteria['confidence_threshold'] and
            reflection['completeness_score'] >= self.validation_criteria['completeness_threshold'] and
            not reflection['should_revise']
        )
        
        self.update_stats(reflection['overall_success'])
        self.reflection_history.append(reflection)
        
        print(f"   📊 Confidence: {reflection['confidence_score']:.2f}")
        print(f"   📈 Completeness: {reflection['completeness_score']:.2f}")
        print(f"   🎯 Overall Success: {reflection['overall_success']}")
        
        if reflection['issues_found']:
            print(f"   ⚠️  Issues: {len(reflection['issues_found'])}")
        
        return reflection
    
    def _assess_consistency(self, tasks: List[Task]) -> float:
        """Assess consistency of results across tasks"""
        if len(tasks) < 2:
            return 1.0
        
        # Simple consistency check based on confidence variance
        confidence_scores = [t.confidence for t in tasks]
        confidence_std = np.std(confidence_scores)
        
        # Lower standard deviation = higher consistency
        consistency = max(0.0, 1.0 - confidence_std)
        return consistency
    
    def _assess_relevance(self, goal: str, tasks: List[Task]) -> float:
        """Assess relevance of task results to the goal"""
        goal_words = set(goal.lower().split())
        relevance_scores = []
        
        for task in tasks:
            if task.result and isinstance(task.result, str):
                result_words = set(str(task.result).lower().split())
                common_words = goal_words.intersection(result_words)
                relevance = len(common_words) / max(len(goal_words), 1)
                relevance_scores.append(relevance)
            else:
                relevance_scores.append(0.5)  # Neutral for non-text results
        
        return np.mean(relevance_scores) if relevance_scores else 0.5
    
    def _check_validation_criteria(self, reflection: Dict[str, Any]) -> List[str]:
        """Check if reflection meets validation criteria"""
        issues = []
        
        if reflection['confidence_score'] < self.validation_criteria['confidence_threshold']:
            issues.append(f"Low confidence score: {reflection['confidence_score']:.2f}")
        
        if reflection['completeness_score'] < self.validation_criteria['completeness_threshold']:
            issues.append(f"Low completeness: {reflection['completeness_score']:.2f}")
        
        if reflection['consistency_score'] < self.validation_criteria['consistency_threshold']:
            issues.append(f"Low consistency: {reflection['consistency_score']:.2f}")
        
        if reflection['relevance_score'] < self.validation_criteria['relevance_threshold']:
            issues.append(f"Low relevance: {reflection['relevance_score']:.2f}")
        
        return issues
    
    def _generate_recommendations(self, reflection: Dict[str, Any], failed_tasks: List[Task]) -> List[str]:
        """Generate actionable recommendations"""
        recommendations = []
        
        if reflection['confidence_score'] < 0.7:
            recommendations.append("Consider using alternative tools or approaches for higher confidence")
        
        if reflection['completeness_score'] < 0.8:
            recommendations.append("Add more comprehensive tasks to improve completeness")
        
        if failed_tasks:
            recommendations.append(f"Revise {len(failed_tasks)} failed tasks with alternative approaches")
        
        if reflection['relevance_score'] < 0.6:
            recommendations.append("Focus tasks more specifically on the main goal")
        
        if not recommendations:
            recommendations.append("Plan execution appears satisfactory")
        
        return recommendations
    
    def _should_revise_plan(self, reflection: Dict[str, Any]) -> bool:
        """Determine if plan should be revised"""
        # Revise if multiple criteria are not met
        failing_criteria = len(reflection['issues_found'])
        
        if failing_criteria >= 2:  # Two or more issues
            return True
        
        if reflection['confidence_score'] < 0.4:  # Very low confidence
            return True
        
        if reflection['completeness_score'] < 0.5:  # Very low completeness
            return True
        
        return False
    
    def get_reflection_summary(self) -> Dict[str, Any]:
        """Get summary of all reflections"""
        if not self.reflection_history:
            return {'total_reflections': 0}
        
        successful_reflections = [r for r in self.reflection_history if r['overall_success']]
        
        return {
            'total_reflections': len(self.reflection_history),
            'successful_reflections': len(successful_reflections),
            'success_rate': len(successful_reflections) / len(self.reflection_history),
            'avg_confidence': np.mean([r['confidence_score'] for r in self.reflection_history]),
            'avg_completeness': np.mean([r['completeness_score'] for r in self.reflection_history]),
            'revisions_recommended': sum(1 for r in self.reflection_history if r['should_revise'])
        }

# Test reflector
reflector = SelfReflectorAgent()
test_reflection = reflector.process(test_plan)

print(f"✅ Self-Reflector Agent ready!")
print(f"   Test reflection success: {test_reflection['overall_success']}")
print(f"   Test recommendations: {len(test_reflection['recommendations'])}")

🤔 Reflecting on plan execution...
✅ Self-Reflector Agent ready!
   Test reflection success: False
   Test recommendations: 0


## 🎯 Coordinator Agent

Master agent that orchestrates the entire agentic system:

In [9]:
class CoordinatorAgent(BaseAgent):
    def __init__(self):
        super().__init__("Coordinator", AgentType.COORDINATOR)
        
        # Initialize sub-agents
        self.planner = ActionPlannerAgent()
        self.executor = ToolExecutorAgent()
        self.reflector = SelfReflectorAgent()
        
        # Configuration
        self.max_iterations = 3
        self.max_revisions = 2
        
        # LLM for answer synthesis
        self.has_llm = False
        api_key = os.getenv('GEMINI_API_KEY')
        if api_key:
            try:
                genai.configure(api_key=api_key)
                self.model = genai.GenerativeModel('gemini-1.5-flash')
                self.has_llm = True
                print("🤖 Gemini API configured for answer synthesis")
            except Exception as e:
                print(f"⚠️ Gemini error: {e}")
        else:
            print("⚠️ No Gemini API key. Using template synthesis.")
    
    def process(self, query: str) -> AgenticResponse:
        """Main coordination process"""
        print(f"\n🎯 Coordinator processing: '{query}'")
        print("=" * 60)
        
        start_time = time.time()
        iterations = 0
        current_plan = None
        final_reflection = None
        reasoning_steps = []
        tool_usage = {tool_type.value: 0 for tool_type in ToolType}
        
        # Step 1: Initial Planning
        print("📋 Step 1: Creating execution plan...")
        current_plan = self.planner.process(query)
        reasoning_steps.append(f"Created plan with {len(current_plan.tasks)} tasks")
        
        # Iterative execution and refinement
        while iterations < self.max_iterations:
            iterations += 1
            print(f"\n🔄 Iteration {iterations}")
            
            # Step 2: Execute Plan
            print(f"⚡ Step 2: Executing plan (Iteration {iterations})...")
            executed_plan = self.executor.execute_plan(current_plan)
            
            # Update tool usage statistics
            for task in executed_plan.tasks:
                if task.status == PlanStatus.COMPLETED:
                    tool_usage[task.tool_required.value] += 1
            
            reasoning_steps.append(f"Iteration {iterations}: Executed {len(executed_plan.tasks)} tasks")
            
            # Step 3: Reflection and Validation
            print(f"🤔 Step 3: Reflecting on results (Iteration {iterations})...")
            reflection = self.reflector.process(executed_plan)
            final_reflection = reflection
            
            reasoning_steps.append(
                f"Iteration {iterations}: Reflection - Success: {reflection['overall_success']}, "
                f"Confidence: {reflection['confidence_score']:.2f}"
            )
            
            # Check if we should continue iterating
            if reflection['overall_success'] and not reflection['should_revise']:
                print(f"✅ Plan successful! Breaking iteration loop.")
                current_plan = executed_plan
                break
            
            # Step 4: Plan Revision (if needed and possible)
            if reflection['should_revise'] and current_plan.revised_count < self.max_revisions:
                print(f"🔄 Step 4: Revising plan based on reflection...")
                
                failed_tasks = [t for t in executed_plan.tasks if t.status == PlanStatus.FAILED]
                if failed_tasks:
                    # Revise plan with failed task
                    current_plan = self.planner.revise_plan(
                        executed_plan, failed_tasks[0], "Task failed during execution"
                    )
                    reasoning_steps.append(f"Iteration {iterations}: Revised plan due to failed tasks")
                else:
                    # No failed tasks but reflection suggests revision
                    print(f"   ⚠️ Reflection suggests revision but no failed tasks found")
                    current_plan = executed_plan
                    break
            else:
                print(f"   ⚠️ Maximum revisions reached or revision not recommended")
                current_plan = executed_plan
                break
        
        # Step 5: Synthesize Final Answer
        print(f"\n🧠 Step 5: Synthesizing final answer...")
        final_answer = self._synthesize_answer(query, current_plan, final_reflection)
        reasoning_steps.append("Synthesized final answer from execution results")
        
        # Calculate final metrics
        processing_time = time.time() - start_time
        confidence_score = final_reflection['confidence_score'] if final_reflection else 0.0
        
        # Create response
        response = AgenticResponse(
            query=query,
            final_answer=final_answer,
            execution_plan=current_plan,
            tool_usage=tool_usage,
            confidence_score=confidence_score,
            reasoning_steps=reasoning_steps,
            processing_time=processing_time,
            iterations=iterations
        )
        
        self.update_stats(final_reflection['overall_success'] if final_reflection else False)
        
        print(f"\n🎉 Processing complete!")
        print(f"   Iterations: {iterations}")
        print(f"   Processing Time: {processing_time:.2f}s")
        print(f"   Final Confidence: {confidence_score:.2f}")
        
        return response
    
    def _synthesize_answer(self, query: str, plan: Plan, reflection: Dict[str, Any]) -> str:
        """Synthesize final answer from execution results"""
        
        if self.has_llm:
            return self._synthesize_with_llm(query, plan, reflection)
        else:
            return self._synthesize_with_template(query, plan, reflection)
    
    def _synthesize_with_llm(self, query: str, plan: Plan, reflection: Dict[str, Any]) -> str:
        """Use LLM for sophisticated answer synthesis"""
        
        # Prepare context from execution results
        context_parts = []
        for task in plan.tasks:
            if task.status == PlanStatus.COMPLETED and task.result:
                context_parts.append(
                    f"Task: {task.description}\n"
                    f"Tool Used: {task.tool_required.value}\n"
                    f"Result: {str(task.result)[:500]}...\n"
                    f"Confidence: {task.confidence:.2f}\n"
                )
        
        context = "\n" + "="*50 + "\n".join(context_parts)
        
        # Create synthesis prompt
        prompt = f"""You are an AI assistant that synthesizes information from multiple sources to provide comprehensive answers.

Original Query: {query}

Execution Context:
- Plan executed with {len(plan.tasks)} tasks
- {len([t for t in plan.tasks if t.status == PlanStatus.COMPLETED])} tasks completed successfully
- Overall confidence: {reflection['confidence_score']:.2f}
- Plan revised {plan.revised_count} times

Task Results:
{context}

Please synthesize a comprehensive, accurate answer to the original query based on the execution results above. 
Focus on providing value and directly addressing what the user asked.

Answer:"""
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error in LLM synthesis: {str(e)}. Falling back to template synthesis."
    
    def _synthesize_with_template(self, query: str, plan: Plan, reflection: Dict[str, Any]) -> str:
        """Template-based answer synthesis"""
        
        completed_tasks = [t for t in plan.tasks if t.status == PlanStatus.COMPLETED]
        
        if not completed_tasks:
            return f"I apologize, but I was unable to gather sufficient information to answer your question: '{query}'. All execution tasks failed."
        
        answer_parts = []
        answer_parts.append(f"Based on my analysis of '{query}', here's what I found:\n")
        
        # Include results from completed tasks
        for i, task in enumerate(completed_tasks[:3], 1):  # Limit to top 3 results
            if task.result:
                result_text = str(task.result)
                if len(result_text) > 300:
                    result_text = result_text[:300] + "..."
                
                answer_parts.append(f"{i}. **{task.task_type.value.title()} Results** (Tool: {task.tool_required.value}):")
                answer_parts.append(f"   {result_text}")
                answer_parts.append(f"   (Confidence: {task.confidence:.2f})\n")
        
        # Add summary
        answer_parts.append(f"**Summary**: Completed {len(completed_tasks)} out of {len(plan.tasks)} planned tasks with an overall confidence of {reflection['confidence_score']:.2f}.")
        
        if plan.revised_count > 0:
            answer_parts.append(f" The plan was revised {plan.revised_count} times to improve results.")
        
        return "\n".join(answer_parts)
    
    def get_system_status(self) -> Dict[str, Any]:
        """Get comprehensive system status"""
        return {
            'coordinator': self.get_info(),
            'planner': self.planner.get_info(),
            'executor': self.executor.get_info(),
            'reflector': self.reflector.get_info(),
            'tool_stats': self.executor.get_tool_stats(),
            'execution_summary': self.executor.get_execution_summary(),
            'reflection_summary': self.reflector.get_reflection_summary()
        }

# Initialize coordinator
coordinator = CoordinatorAgent()
print("🎯 Coordinator Agent ready with all sub-agents!")
print(f"   Planner: {coordinator.planner.name}")
print(f"   Executor: {coordinator.executor.name}")
print(f"   Reflector: {coordinator.reflector.name}")
print(f"   Available Tools: {len(coordinator.executor.tools)}")

🤖 Gemini API configured for answer synthesis
🎯 Coordinator Agent ready with all sub-agents!
   Planner: ActionPlanner
   Executor: ToolExecutor
   Reflector: SelfReflector
   Available Tools: 5


## 🤖 Complete Agentic RAG System

Integrate everything into the complete autonomous system:

In [10]:
class AgenticRAGSystem:
    def __init__(self):
        print("🤖 Initializing Agentic RAG System...")
        
        # Initialize the coordinator (which manages all other agents)
        self.coordinator = CoordinatorAgent()
        
        # System configuration
        self.config = {
            'max_query_length': 500,
            'timeout_seconds': 120,
            'enable_parallel_execution': False,  # Future enhancement
            'enable_learning': False,  # Future enhancement
        }
        
        # System statistics
        self.total_queries = 0
        self.successful_queries = 0
        self.total_processing_time = 0.0
        self.query_history = []
        
        print("✅ Agentic RAG System initialized!")
        print(f"🎯 Coordinator: {self.coordinator.name}")
        print(f"🛠️ Available Tools: {len(self.coordinator.executor.tools)}")
        print(f"📊 Max Iterations: {self.coordinator.max_iterations}")
    
    def solve(self, query: str, user_id: str = None) -> AgenticResponse:
        """Main entry point for solving queries"""
        
        # Validate input
        if not query or len(query.strip()) == 0:
            raise ValueError("Query cannot be empty")
        
        if len(query) > self.config['max_query_length']:
            raise ValueError(f"Query too long. Maximum length: {self.config['max_query_length']}")
        
        # Process query
        self.total_queries += 1
        start_time = time.time()
        
        try:
            print(f"\n{'='*80}")
            print(f"🤖 AGENTIC RAG SYSTEM - SOLVING QUERY")
            print(f"{'='*80}")
            print(f"📝 Query: {query}")
            print(f"👤 User: {user_id or 'Anonymous'}")
            print(f"🕐 Started: {datetime.now().strftime('%H:%M:%S')}")
            
            # Delegate to coordinator
            response = self.coordinator.process(query)
            
            # Update statistics
            processing_time = time.time() - start_time
            self.total_processing_time += processing_time
            
            if response.confidence_score > 0.5:  # Threshold for success
                self.successful_queries += 1
            
            # Store in history
            query_record = {
                'query': query,
                'user_id': user_id,
                'timestamp': datetime.now(),
                'processing_time': processing_time,
                'confidence': response.confidence_score,
                'iterations': response.iterations,
                'tools_used': sum(response.tool_usage.values()),
                'success': response.confidence_score > 0.5
            }
            self.query_history.append(query_record)
            
            print(f"\n🎉 QUERY SOLVED SUCCESSFULLY!")
            print(f"{'='*80}")
            
            return response
            
        except Exception as e:
            processing_time = time.time() - start_time
            self.total_processing_time += processing_time
            
            # Create error response
            error_response = AgenticResponse(
                query=query,
                final_answer=f"I encountered an error while processing your query: {str(e)}",
                execution_plan=Plan("error", query, []),
                tool_usage={tool_type.value: 0 for tool_type in ToolType},
                confidence_score=0.0,
                reasoning_steps=[f"Error occurred: {str(e)}"],
                processing_time=processing_time,
                iterations=0
            )
            
            print(f"\n❌ ERROR OCCURRED: {str(e)}")
            print(f"{'='*80}")
            
            return error_response
    
    def get_system_metrics(self) -> Dict[str, Any]:
        """Get comprehensive system performance metrics"""
        
        avg_processing_time = (
            self.total_processing_time / max(self.total_queries, 1)
        )
        success_rate = (
            self.successful_queries / max(self.total_queries, 1) * 100
        )
        
        recent_queries = self.query_history[-10:] if self.query_history else []
        
        return {
            'system_overview': {
                'total_queries': self.total_queries,
                'successful_queries': self.successful_queries,
                'success_rate': success_rate,
                'avg_processing_time': avg_processing_time,
                'total_processing_time': self.total_processing_time
            },
            'agent_performance': self.coordinator.get_system_status(),
            'recent_activity': recent_queries,
            'configuration': self.config
        }
    
    def get_performance_summary(self) -> str:
        """Get human-readable performance summary"""
        metrics = self.get_system_metrics()
        overview = metrics['system_overview']
        
        summary = f"""
🤖 AGENTIC RAG SYSTEM PERFORMANCE SUMMARY
{'='*50}

📊 Overall Performance:
   • Total Queries Processed: {overview['total_queries']}
   • Successful Queries: {overview['successful_queries']}
   • Success Rate: {overview['success_rate']:.1f}%
   • Average Processing Time: {overview['avg_processing_time']:.2f}s

🤖 Agent Performance:
   • Coordinator Success Rate: {self.coordinator.get_success_rate()*100:.1f}%
   • Planner Success Rate: {self.coordinator.planner.get_success_rate()*100:.1f}%
   • Executor Success Rate: {self.coordinator.executor.get_success_rate()*100:.1f}%
   • Reflector Success Rate: {self.coordinator.reflector.get_success_rate()*100:.1f}%

🛠️ Tool Usage:
"""
        
        tool_stats = metrics['agent_performance']['tool_stats']
        for tool_name, stats in tool_stats.items():
            usage_count = stats['usage_count']
            success_rate = stats['tool_stats']['success_rate'] * 100
            summary += f"   • {tool_name}: {usage_count} uses ({success_rate:.1f}% success)\n"
        
        return summary

# Initialize the complete system
agentic_rag = AgenticRAGSystem()
print("\n🚀 Complete Agentic RAG System ready!")

🤖 Initializing Agentic RAG System...
🤖 Gemini API configured for answer synthesis
✅ Agentic RAG System initialized!
🎯 Coordinator: Coordinator
🛠️ Available Tools: 5
📊 Max Iterations: 3

🚀 Complete Agentic RAG System ready!


## 🧪 Comprehensive Testing Suite

Test the agentic system with various query types:

In [11]:
def run_agentic_test_suite():
    print("\n" + "🧪"*20 + " AGENTIC RAG TEST SUITE " + "🧪"*20)
    
    test_cases = [
        {
            "name": "Simple Information Query",
            "query": "What is machine learning?",
            "expected_tools": ["web_search", "retrieval"],
            "user_id": "test_user_1"
        },
        {
            "name": "Mathematical Calculation",
            "query": "Calculate the compound interest on $1000 at 5% annually for 3 years",
            "expected_tools": ["calculator"],
            "user_id": "test_user_2"
        },
        {
            "name": "Research and Analysis",
            "query": "Analyze the impact of quantum computing on current encryption methods",
            "expected_tools": ["web_search", "retrieval", "analyzer"],
            "user_id": "test_user_3"
        },
        {
            "name": "Database Query",
            "query": "Find information about technology companies founded after 1990",
            "expected_tools": ["database"],
            "user_id": "test_user_4"
        },
        {
            "name": "Complex Multi-Step Query",
            "query": "Compare the revenue growth of Apple and Microsoft, then calculate the percentage difference",
            "expected_tools": ["database", "calculator", "analyzer"],
            "user_id": "test_user_5"
        }
    ]
    
    results = []
    total_start_time = time.time()
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\n{'='*70}")
        print(f"🔬 TEST CASE {i}: {test_case['name']}")
        print(f"❓ Query: '{test_case['query']}'")
        print(f"👤 User: {test_case['user_id']}")
        print("=" * 70)
        
        try:
            # Execute test
            response = agentic_rag.solve(
                test_case['query'], 
                test_case['user_id']
            )
            
            # Analyze results
            tools_used = [tool for tool, count in response.tool_usage.items() if count > 0]
            expected_tools = test_case['expected_tools']
            tools_match = set(tools_used) == set(expected_tools)
            
            test_result = {
                "test_case": test_case['name'],
                "query": test_case['query'],
                "success": response.confidence_score > 0.5,
                "confidence": response.confidence_score,
                "iterations": response.iterations,
                "processing_time": response.processing_time,
                "tools_used": tools_used,
                "expected_tools": expected_tools,
                "tools_match": tools_match,
                "response": response
            }
            
            results.append(test_result)
            
            print(f"\n📊 TEST RESULTS:")
            print(f"   ✅ Success: {test_result['success']}")
            print(f"   🎯 Confidence: {test_result['confidence']:.2f}")
            print(f"   🔄 Iterations: {test_result['iterations']}")
            print(f"   ⏱️ Processing Time: {test_result['processing_time']:.2f}s")
            print(f"   🛠️ Tools Used: {', '.join(test_result['tools_used'])}")
            print(f"   🧰 Expected Tools: {', '.join(test_result['expected_tools'])}")
            print(f"   🔧 Tools Match: {test_result['tools_match']}")
            
        except Exception as e:
            print(f"❌ Test failed with error: {str(e)}")
            results.append({
                "test_case": test_case['name'],
                "error": str(e),
                "success": False
            })
    
    total_time = time.time() - total_start_time
    
    # Print summary
    print(f"\n{'='*70}")
    print("📈 TEST SUITE SUMMARY")
    print("="*70)
    
    passed = sum(1 for r in results if r.get('success', False))
    failed = len(results) - passed
    
    print(f"   ✅ Passed: {passed}")
    print(f"   ❌ Failed: {failed}")
    print(f"   ⏱️ Total Time: {total_time:.2f}s")
    print(f"   🎯 Average Confidence: {np.mean([r.get('confidence', 0) for r in results if 'confidence' in r]):.2f}")
    
    # Print detailed results
    print("\n🔍 DETAILED RESULTS:")
    for i, result in enumerate(results, 1):
        print(f"\nTest {i}: {result['test_case']}")
        print(f"   Query: {result['query']}")
        if 'success' in result:
            print(f"   Status: {'✅ PASSED' if result['success'] else '❌ FAILED'}")
            print(f"   Confidence: {result.get('confidence', 0):.2f}")
            print(f"   Tools Used: {', '.join(result.get('tools_used', []))}")
            print(f"   Expected Tools: {', '.join(result.get('expected_tools', []))}")
            print(f"   Tools Match: {result.get('tools_match', False)}")
        else:
            print(f"   Error: {result.get('error', 'Unknown error')}")
    
    return results

# Run the test suite
test_results = run_agentic_test_suite()

# Print system performance after testing
print("\n" + "📊"*20 + " SYSTEM PERFORMANCE " + "📊"*20)
print(agentic_rag.get_performance_summary())


🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪 AGENTIC RAG TEST SUITE 🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪

🔬 TEST CASE 1: Simple Information Query
❓ Query: 'What is machine learning?'
👤 User: test_user_1

🤖 AGENTIC RAG SYSTEM - SOLVING QUERY
📝 Query: What is machine learning?
👤 User: test_user_1
🕐 Started: 12:01:24

🎯 Coordinator processing: 'What is machine learning?'
📋 Step 1: Creating execution plan...

🔄 Iteration 1
⚡ Step 2: Executing plan (Iteration 1)...
🎯 Executing plan: What is machine learning?
⚡ Executing: Search for information about: What is machine learning?
   ✅ Completed in 0.00s (Confidence: 0.80)
⚡ Executing: Analyze gathered information about: What is machine learning?
   ✅ Completed in 0.00s (Confidence: 0.80)
⚡ Executing: Synthesize final answer for: What is machine learning?
   ✅ Completed in 0.02s (Confidence: 0.90)
🤔 Step 3: Reflecting on results (Iteration 1)...
🤔 Reflecting on plan execution...
   📊 Confidence: 0.83
   📈 Completeness: 1.00
   🎯 Overall Success: True
   ⚠️  Issues: 1
✅ Plan successfu